In [1]:
import pandas as pd
import numpy as np
from keras import backend as K
from keras.models import Sequential
from keras import losses
from keras.layers import Dense, Activation
from sklearn import datasets
from sklearn.preprocessing import OneHotEncoder

Using TensorFlow backend.


## 讀取資料
* 將資料分為輸入特徵及類別
* 將多分類問題轉為二元分類

In [2]:
iris = datasets.load_iris()
X = iris.data
Y = iris.target
Y[Y == 2] = 1
Y = Y.reshape(-1, 1)

## 共軛梯度涵式
* 建立神經網路
* 取得網路權重
* 設定網路權重
* 取得當前梯度
* 進行梯度壓縮
* 還原壓縮梯度
* 取得解空間的長度
* 更新網路權重

In [3]:
# 建立神經網路

def create_model():
    model = Sequential()
    model.add(Dense(units = 5, activation = 'relu', input_dim = X.shape[1])) 
    model.add(Dense(units = 1, activation = 'sigmoid'))
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam')
    
    return(model)

# 取得網路權重

def get_weight(model):
    return(model.get_weights())

# 設定網路權重

def set_weight(model, weights):
    return(model.set_weights(weights))

# 取得當前梯度

def get_weight_grad(model, inputs, outputs):
    grads = model.optimizer.get_gradients(model.total_loss, model.trainable_weights)
    symb_inputs = (model._feed_inputs + model._feed_targets + model._feed_sample_weights)
    f = K.function(symb_inputs, grads)
    x, y, sample_weight = model._standardize_user_data(inputs, outputs)
    output_grad = f(x + y + sample_weight)
    return output_grad

# 進行梯度壓縮

def flatten_grad(grad):
    shape_list = list()
    range_list = list()
    value_list = list()
    
    for i in range(len(grad)):
        shape_list.append(np.array(grad[i]).shape)
        if(i == 0):
            range_list.append(np.array(grad[i]).reshape(-1).shape[0])
        else:
            range_list.append(np.array(grad[i]).reshape(-1).shape[0] + range_list[i - 1])
            
    for i in range(len(grad)):
        value_list = value_list + np.array(grad[i]).reshape(-1).tolist()
        
    return shape_list, range_list, value_list

# 還原壓縮梯度

def construct_grad(shape_list, range_list, value_list):
    grad = list()
    for i in range(len(shape_list)):
        beg = 0
        end = range_list[i]
        if(i > 0):
            beg = range_list[i - 1]

        grad.append(np.array(value_list[beg:end]).reshape(shape_list[i]))
        
    return(grad)

# 取得解空間的長度

def get_weight_length(value_list):
    length = 0
    
    for i in range(len(value_list)):
        length = length + np.power(value_list[i], 2)
    
    return(np.sqrt(length))

# 更新網路權重

def step(weight, grad):
    update_weight = weight
    for i in range(len(weight)):
        update_weight[i] = weight[i] + grad[i]
    
    return(update_weight)

## 共軛伸縮梯度下降步驟
* 步驟1: 初始化
* 步驟2: 計算二階梯度資訊
* 步驟3: 進行 delta k 伸縮
* 步驟4: 如果 delta_k <= 0, 讓 hessian 轉為正定
* 步驟5: 計算移動步長 alpha
* 步驟6: 計算比較參數 comp
* 步驟7: 若 comp >= 0, 表示損失函數可以降低
* 步驟8: 若 comp < 0.25, 增加伸縮尺度 scale
* 步驟9: 若梯度等於 0, 表示最佳解尋得, 停止演算法

In [15]:
def scaled_conjugate_gradient(model, inputs, outputs, maxiter):
    
    # 步驟1: 初始化

    sigma = 1e-4
    lambda_ = 1e-6
    lambda_hat = 0
    p = [(grad * -1) for grad in get_weight_grad(model, X, Y)]
    p_prior = p
    shape_list, range_list, value_list = flatten_grad(p)
    p_flatten = np.array(value_list)
    p_flatten_prior = p_flatten
    r = p_flatten
    r_prior = r
    N = len(value_list)
    success = True

    for k in range(1, maxiter + 1):

        # 步驟2: 計算二階梯度資訊

        model_appr = create_model()
        sigma_k = sigma / get_weight_length(p_flatten)
        set_weight(model_appr, step(get_weight(model), [grad * sigma_k for grad in p]))

        grad_appr = get_weight_grad(model_appr, X, Y)
        grad_curr = get_weight_grad(model, X, Y)
        _shape_list, _range_list, grad_appr = np.array(flatten_grad(grad_appr))
        _shape_list, _range_list, grad_curr = np.array(flatten_grad(grad_curr))
        grad_appr = np.array(grad_appr)
        grad_curr = np.array(grad_curr)

        s = (grad_appr - grad_curr) / sigma_k
        delta = np.dot(p_flatten, s)

        # 步驟3: 進行 delta k 伸縮

        delta = delta + (lambda_ - lambda_hat) * np.dot(p_flatten, p_flatten)

        # 步驟4: 如果 delta_k <= 0, 讓 hessian 轉為正定

        if(delta <= 0):
            lambda_hat = 2 * (lambda_ - (delta / np.dot(p_flatten, p_flatten)))
            delta = -delta + (lambda_ * np.dot(p_flatten, p_flatten))
            lambda_ = lambda_hat

        # 步驟5: 計算移動步長 alpha

        mu = np.dot(p_flatten, r)
        alpha = mu / delta

        # 步驟6: 計算比較參數 comp

        set_weight(model_appr, step(get_weight(model), [grad * alpha for grad in p]))
        loss_curr = model.evaluate(X, Y)
        loss_appr = model_appr.evaluate(X, Y)
        comp = 2 * (loss_curr - loss_appr) / np.power(mu, 2)

        # 步驟7: 若 comp >= 0, 表示損失函數可以降低

        if(comp >= 0):
            set_weight(model, step(get_weight(model), [grad * alpha for grad in p]))
            r_prior = r
            r = [(grad * -1) for grad in get_weight_grad(model, X, Y)]
            lambda_hat = 0
            success = True

            if(k % N == 0):
                p_prior = p
                p = r
                shape_list, range_list, r = flatten_grad(r)
                p_flatten = r
            else:
                shape_list, range_list, r = flatten_grad(r)
                beta = ((np.dot(r, r) - np.dot(r, r_prior)) / mu)
                p_prior = p
                p = step(construct_grad(shape_list, range_list, r), [(weight * beta) for weight in p])
                p_flatten_prior = p_flatten
                shape_list, range_list, p_flatten = flatten_grad(p)

            if(comp >= 0.75):
                lambda_ = lambda_ / 4
        else:
            lambda_hat = lambda_
            success = False

        # 步驟8: 若 comp < 0.25, 增加伸縮尺度 scale

        if(comp < 0.25):
            lambda_ = lambda_ + (delta * (1 - comp) / np.dot(p_flatten_prior, p_flatten_prior))

        # 步驟9: 若梯度等於 0, 表示最佳解尋得, 停止演算法

        print(comp)
        if(np.array_equal(r, np.zeros(len(r)))):
            print('converge to local optima')
            break;

## 建立神經網路模型

In [19]:
model = create_model()

## 使用共軛梯度下降優化網路權重

In [20]:
scaled_conjugate_gradient(model, X, Y, 10)

150/150 [==============================] - 1s 4ms/step
0.621233856561797
150/150 [==============================] - 1s 4ms/step
-59.404584952096556
150/150 [==============================] - 1s 4ms/step
4.271329610137764
150/150 [==============================] - 1s 4ms/step
5.657324350621068
150/150 [==============================] - 1s 5ms/step
101.04409448855539
150/150 [==============================] - 1s 5ms/step
0.2993012323401148
150/150 [==============================] - 1s 5ms/step
26.868836975607074
150/150 [==============================] - 1s 5ms/step
271.9907103646248
150/150 [==============================] - 1s 5ms/step
535.0497201084588
150/150 [==============================] - 1s 5ms/step
766.88575004283


## 使用準確率評估模型的表現

In [21]:
print('accuracy: {0}%'.format((sum(np.round(model.predict(X)) == Y) / len(Y)) * 100))

accuracy: [100.]%
